# Titanic Machine Learning

In [1]:
import pandas as pd
import numpy as np

In [11]:
train_data = pd.read_csv('train.csv', index_col='PassengerId')
test_data = pd.read_csv('test.csv', index_col='PassengerId')

In [39]:
train_data

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [16]:
# Treating Data
train_data.loc[train_data.loc[:,'Name'].str.lower().str.contains('jack'),:]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
767,0,1,"Brewe, Dr. Arthur Jackson",male,NaN,0,0,112379,39.6,NaN,C


In [17]:
train_data.count()

Survived    891
Pclass      891
Name        891
Sex         891
Age         714
SibSp       891
Parch       891
Ticket      891
Fare        891
Cabin       204
Embarked    889
dtype: int64

In [40]:
y = train_data.loc[:,'Survived'].values

In [63]:
x_col = ['Sex','Pclass','Age','SibSp','Parch','Fare','Embarked']
x_data = train_data.loc[:,x_col].copy()
# Lấy giá trị xảy ra nhiều nhất của embarked
most_freq_embarked = x_data.groupby('Embarked')['Sex'].count().idxmax()
x_data.loc[:,'Embarked'] = x_data.loc[:,'Embarked'].fillna(most_freq_embarked)
x_col = pd.get_dummies(x_data).drop(['Sex_male','Embarked_S'], axis=1).columns
X = pd.get_dummies(x_data).drop(['Sex_male','Embarked_S'], axis=1).values

In [50]:
# Chia train/test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

In [51]:
# Missing Value Imputations
from sklearn.impute import SimpleImputer
imputers = SimpleImputer(strategy = 'median')
X_train = imputers.fit_transform(X_train)
X_test = imputers.transform(X_test)

In [53]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_1 = scaler.fit_transform(X_train)
X_test_1 = scaler.transform(X_test)

In [56]:
# Logistics Classification
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(C = 1.0, solver = 'liblinear')
log_reg.fit(X_train_1, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [65]:
y_pred = log_reg.predict_proba(X_test_1)

In [64]:
display(x_col)
display(log_reg.coef_)

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex_female', 'Embarked_C',
       'Embarked_Q'],
      dtype='object')

array([[-0.94922276, -0.58967727, -0.30448484, -0.0363225 ,  0.10468507,
         1.2669141 ,  0.11781317,  0.17591533]])

In [79]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, (y_pred[:,1]>0.5).astype('i8'))
print('Confusion Matrix: \n', cm)
print('Accuracy Score: ', (cm[0,0] + cm[1,1])/cm.sum())

Confusion Matrix: 
 [[156  26]
 [ 31  82]]
Accuracy Score:  0.8067796610169492


In [82]:
from sklearn.svm import SVC
svmc = SVC(C = 1.0, probability=True)
svmc.fit(X_train_1, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [83]:
y_pred = svmc.predict_proba(X_test_1)

In [84]:
cm = confusion_matrix(y_test, (y_pred[:,1]>0.5).astype('i8'))
print('Confusion Matrix: \n', cm)
print('Accuracy Score: ', (cm[0,0] + cm[1,1])/cm.sum())

Confusion Matrix: 
 [[165  17]
 [ 37  76]]
Accuracy Score:  0.8169491525423729


In [96]:
from sklearn import tree

da_tree = tree.DecisionTreeClassifier(max_depth=3, min_samples_leaf=10 )
da_tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [105]:
from sklearn.model_selection import GridSearchCV
classifier = tree.DecisionTreeClassifier() 
parameters = {"max_depth": [1,2,3,4], 'min_samples_leaf': [(e+1)*5 for e in range(5)] }
gs = GridSearchCV(classifier, param_grid=parameters, cv=3, n_jobs=3, scoring='f1_micro')
gs.fit(X_train, y_train)
print ("BEST", gs.best_params_, gs.best_score_)
clsf = gs.best_estimator_

BEST {'max_depth': 4, 'min_samples_leaf': 5} 0.8087248322147651


In [106]:
y_pred = clsf.predict_proba(X_test)
cm = confusion_matrix(y_test, (y_pred[:,1]>0.5).astype('i8'))
print('Confusion Matrix: \n', cm)
print('Accuracy Score: ', (cm[0,0] + cm[1,1])/cm.sum())

Confusion Matrix: 
 [[169  13]
 [ 49  64]]
Accuracy Score:  0.7898305084745763


In [90]:
import graphviz
def print_tree(clsf, file_name, class_name =[0,1],  feature_cols = None):
    dot_data = tree.export_graphviz(clsf, out_file=None, feature_names = feature_cols, class_names = class_name)
    graph = graphviz.Source(dot_data)
    graph.render(file_name)


In [107]:
print_tree(clsf, './tree2', ['J','R'],feature_cols =x_col)

In [100]:
X_train[:,1]

array([ 1.  , 36.  , 12.  , 49.  , 33.  , 30.  , 30.  , 29.  , 22.  ,
       34.  , 29.  , 52.  , 29.  , 19.  , 29.  , 19.  , 22.  , 29.  ,
       29.  , 26.  , 29.  , 29.  , 33.  ,  4.  , 24.  , 47.  , 40.  ,
       29.  , 32.5 , 32.  , 19.  , 16.  ,  1.  , 50.  , 36.  , 44.  ,
       31.  , 19.  , 29.  , 29.  , 55.  , 22.  , 60.  , 22.  , 39.  ,
       51.  ,  1.  , 18.  ,  4.  ,  6.  , 59.  , 20.  , 17.  , 43.  ,
        7.  , 48.  , 24.  , 42.  , 19.  , 17.  , 20.  , 10.  , 21.  ,
       71.  , 18.  , 26.  , 26.  , 71.  , 15.  , 24.  , 33.  , 40.  ,
       29.  , 32.  , 30.5 , 29.  , 16.  , 29.  , 23.  , 22.  , 23.  ,
       30.  , 22.  , 48.  , 21.  , 29.  ,  3.  , 17.  , 54.  , 28.  ,
       29.  , 13.  , 17.  , 19.  , 30.  , 29.  , 26.  , 36.  ,  4.  ,
       24.  , 25.  , 38.  ,  6.  , 24.  , 22.  , 29.  , 29.  , 21.  ,
       29.  , 44.  ,  5.  , 24.  , 38.  , 35.  , 47.  , 29.  , 42.  ,
       16.  , 31.  , 25.  , 57.  ,  0.75, 19.  , 31.  , 18.  , 18.  ,
       24.  , 61.  ,